In [6]:
import math
import numpy as np
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms, datasets
from torch import optim 
from torchvision import models
from torchsummary import summary

In [7]:
class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
path_cifar100 = './cifar100'
train_dataset_cifar100 = datasets.CIFAR100(root=path_cifar100, train=True, download=True)
test_dataset_cifar100 = datasets.CIFAR100(root=path_cifar100, train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


## CNN (самописная) на CIFAR-100.

In [10]:
train_actions = transforms.ToTensor()
test_transforms = transforms.ToTensor()

In [11]:
train_dataset = MyOwnCifar(train_dataset_cifar100, train_actions)
test_dataset = MyOwnCifar(test_dataset_cifar100, test_transforms)

In [12]:
loader_train = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=128, 
                                           shuffle=True)
loader_test = torch.utils.data.DataLoader(test_dataset, batch_size=128,
                                          shuffle=False)

In [13]:
class Net(nn.Module):
    def __init__(self, device='cpu'):
        super().__init__()
        self.net = nn.Sequential(
          nn.BatchNorm2d(3), 
          nn.Conv2d(3, 64, 3),
          nn.LeakyReLU(),
          nn.BatchNorm2d(64), 
          nn.Conv2d(64, 64, 3),
          nn.LeakyReLU(),
          nn.BatchNorm2d(64), 
          nn.MaxPool2d(2, 2), #16
          nn.Conv2d(64, 128, 3),
          nn.LeakyReLU(),          
          nn.BatchNorm2d(128), 
          nn.Conv2d(128, 128, 3),
          nn.LeakyReLU(),          
          nn.BatchNorm2d(128),                     
          nn.MaxPool2d(2, 2), #16
          nn.Conv2d(128, 256, 3),
          nn.LeakyReLU(),
          nn.BatchNorm2d(256),           
          nn.Conv2d(256, 512, 3),
          nn.Flatten(),
          nn.Linear(512, 1024),
          nn.LeakyReLU(),    
          nn.Dropout(0.2),
          nn.BatchNorm1d(1024),                 
          nn.Linear(1024, 1024),
          nn.LeakyReLU(), 
          nn.Dropout(0.2),             
          nn.BatchNorm1d(1024),                    
          nn.Linear(1024, 100), 
          nn.Softmax(dim=1)
        )
        self.mean_target_train = 0
        self.mean_target_test = 0
        self.history = None
        self.device=device

        
    def forward(self, x):
        x = self.net(x)
        return x

    
    def fit(self, loader_train, loadre_test, epochs, loss, optimazer, lr, show_log=True):
        self.train()
        self.history = []
        self.optimazer = optimazer(self.parameters(), lr)
        self.loss = loss()
        for epoch in range(epochs):
            running_loss = 0.0
            running_items = 0.0
            predict_ture = 0.0            
            for i, data in enumerate(loader_train):
                X_train, y_train = data[0].to(self.device), data[1].to(self.device)
                self.optimazer.zero_grad()
                outputs = self(X_train)
                loss_iter = self.loss(outputs, y_train)
                loss_iter.backward()
                self.optimazer.step()
                if i%200 ==0:
                    loss_train, acc_train = self.loss_fn(loader_train)
                    loss_test, acc_test = self.loss_fn(loader_test)
                    self.history.append([epoch+1, i+1, acc_train, acc_test])
                    if show_log:
                        print(f'Epoch [{epoch + 1:2}/{epochs}]. ' \
                              f'Step [{i + 1:3}/{len(loader_train)}]. ' \
                              f'Loss_valid: {loss_train:.4f}  '\
                              f'Acc_valid: {acc_train:.3f}  '\
                              f'Loss_test: {loss_test:.4f}  '\
                              f'Acc_test: {acc_test:.3f}'
                              )
                running_loss, running_items = 0.0, 0.0
        print(f'Training is finished with optimazer: {optimazer}')
        return self.history

    
    def loss_fn(self, loader):
        self.eval()
        running_loss = 0.0
        running_items = 0.0
        predict_ture = 0.0        
        for i, data in enumerate(loader):
            X_data, y_data = data[0].to(self.device), data[1].to(self.device)
            outputs = self(X_data)
            loss_iter = self.loss(outputs, y_data)
            running_loss += loss_iter.item()
            _, predicted = torch.max(outputs, 1)
            predict_ture += (y_data==predicted).sum().item()
            running_items += len(y_data)
        self.train()  
        return running_loss / running_items, predict_ture/running_items
  

In [14]:
net = Net(device).to(device)

In [15]:
summary(net.to(device), input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1            [-1, 3, 32, 32]               6
            Conv2d-2           [-1, 64, 30, 30]           1,792
         LeakyReLU-3           [-1, 64, 30, 30]               0
       BatchNorm2d-4           [-1, 64, 30, 30]             128
            Conv2d-5           [-1, 64, 28, 28]          36,928
         LeakyReLU-6           [-1, 64, 28, 28]               0
       BatchNorm2d-7           [-1, 64, 28, 28]             128
         MaxPool2d-8           [-1, 64, 14, 14]               0
            Conv2d-9          [-1, 128, 12, 12]          73,856
        LeakyReLU-10          [-1, 128, 12, 12]               0
      BatchNorm2d-11          [-1, 128, 12, 12]             256
           Conv2d-12          [-1, 128, 10, 10]         147,584
        LeakyReLU-13          [-1, 128, 10, 10]               0
      BatchNorm2d-14          [-1, 128,

In [16]:
hist = net.fit(loader_train, loader_test, 5, nn.CrossEntropyLoss, optim.Adam, 0.0001, show_log=True)

Epoch [ 1/5]. Step [  1/391]. Loss_valid: 0.0360  Acc_valid: 0.010  Loss_test: 0.0364  Acc_test: 0.010
Epoch [ 1/5]. Step [201/391]. Loss_valid: 0.0353  Acc_valid: 0.107  Loss_test: 0.0357  Acc_test: 0.103
Epoch [ 2/5]. Step [  1/391]. Loss_valid: 0.0351  Acc_valid: 0.139  Loss_test: 0.0355  Acc_test: 0.135
Epoch [ 2/5]. Step [201/391]. Loss_valid: 0.0348  Acc_valid: 0.174  Loss_test: 0.0353  Acc_test: 0.166
Epoch [ 3/5]. Step [  1/391]. Loss_valid: 0.0347  Acc_valid: 0.190  Loss_test: 0.0352  Acc_test: 0.177
Epoch [ 3/5]. Step [201/391]. Loss_valid: 0.0345  Acc_valid: 0.218  Loss_test: 0.0350  Acc_test: 0.202
Epoch [ 4/5]. Step [  1/391]. Loss_valid: 0.0343  Acc_valid: 0.241  Loss_test: 0.0349  Acc_test: 0.216
Epoch [ 4/5]. Step [201/391]. Loss_valid: 0.0342  Acc_valid: 0.257  Loss_test: 0.0348  Acc_test: 0.231
Epoch [ 5/5]. Step [  1/391]. Loss_valid: 0.0340  Acc_valid: 0.283  Loss_test: 0.0346  Acc_test: 0.248
Epoch [ 5/5]. Step [201/391]. Loss_valid: 0.0339  Acc_valid: 0.304  Loss_

## CNN на CIFAR-100 через дообучение ImageNet Resnet-50.

In [17]:
resnet50 = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [18]:
train_actions = transforms.Compose([transforms.Resize(256),
                                    transforms.RandomCrop(224, padding=4), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])

In [19]:
train_dataset = MyOwnCifar(train_dataset_cifar100, train_actions)
test_dataset = MyOwnCifar(test_dataset_cifar100, test_transforms)

In [20]:
loader_train = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=2)
loader_test = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=128,
                                           shuffle=False,
                                           num_workers=1)

In [21]:
class Net_resnet(nn.Module):
    def __init__(self, resnet50, device='cpu'):
        super().__init__()
        self.device = device
        self.net = resnet50
        for param in list(self.net.parameters())[:]:
            param.requires_grad = False
        self.net.fc = nn.Sequential(            
            nn.Linear(2048, 2048),
            nn.LeakyReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.25),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.25),
            nn.Linear(1024, 100),
            nn.Softmax(dim=1)
            )

    def forward(self, x):
        x = self.net(x)
        return x


    def fit(self, loader_train, loadre_test, epochs, loss, optimazer, lr, show_log=True):
        self.train()
        params_to_update = []
        for name, param in self.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
        self.history = []
        self.optimazer = optimazer(params_to_update, lr)    
        self.loss = loss()
        for epoch in range(epochs):
            running_loss = 0.0
            running_items = 0.0
            predict_ture = 0.0            
            for i, data in enumerate(loader_train):
                X_train, y_train = data[0].to(self.device), data[1].to(self.device)
                self.optimazer.zero_grad()
                outputs = self(X_train)
                loss_iter = self.loss(outputs, y_train)
                loss_iter.backward()
                self.optimazer.step()
                if i%200 ==0:
                    loss_train, acc_train = self.loss_fn(loader_train)
                    loss_test, acc_test = self.loss_fn(loader_test)
                    self.history.append([epoch+1, i+1, acc_train, acc_test])
                    if show_log:
                        print(f'Epoch [{epoch + 1:2}/{epochs}]. ' \
                              f'Step [{i + 1:3}/{len(loader_train)}]. ' \
                              f'Loss_valid: {loss_train:.4f}  '\
                              f'Acc_valid: {acc_train:.3f}  '\
                              f'Loss_test: {loss_test:.4f}  '\
                              f'Acc_test: {acc_test:.3f}'
                              )
                running_loss, running_items = 0.0, 0.0
        print(f'Training is finished with optimazer: {optimazer}')
        return self.history


    def loss_fn(self, loader):
        self.eval()
        running_loss = 0.0
        running_items = 0.0
        predict_ture = 0.0    
        for i, data in enumerate(loader):
            X_data, y_data = data[0].to(self.device), data[1].to(self.device)
            outputs = self(X_data)
            loss_iter = self.loss(outputs, y_data)
            running_loss += loss_iter.item()
            _, predicted = torch.max(outputs, 1)
            predict_ture += (y_data==predicted).sum().item()
            running_items += len(y_data)
        self.train()  
        return running_loss / running_items, predict_ture/running_items


In [22]:
net = Net_resnet(resnet50, device).to(device)

In [23]:
summary(net, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,096
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          16,384
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13            [-1, 256, 8, 8]          16,384
      BatchNorm2d-14            [-1, 25

In [24]:
hist = net.fit(loader_train, loader_test, 5, nn.CrossEntropyLoss, optim.Adam, 0.0001, show_log=True)

Epoch [ 1/5]. Step [  1/391]. Loss_valid: 0.0360  Acc_valid: 0.017  Loss_test: 0.0364  Acc_test: 0.016
Epoch [ 1/5]. Step [201/391]. Loss_valid: 0.0336  Acc_valid: 0.374  Loss_test: 0.0340  Acc_test: 0.372
Epoch [ 2/5]. Step [  1/391]. Loss_valid: 0.0331  Acc_valid: 0.445  Loss_test: 0.0334  Acc_test: 0.443
Epoch [ 2/5]. Step [201/391]. Loss_valid: 0.0328  Acc_valid: 0.477  Loss_test: 0.0332  Acc_test: 0.464
Epoch [ 3/5]. Step [  1/391]. Loss_valid: 0.0326  Acc_valid: 0.495  Loss_test: 0.0330  Acc_test: 0.486
Epoch [ 3/5]. Step [201/391]. Loss_valid: 0.0324  Acc_valid: 0.518  Loss_test: 0.0328  Acc_test: 0.505
Epoch [ 4/5]. Step [  1/391]. Loss_valid: 0.0323  Acc_valid: 0.539  Loss_test: 0.0327  Acc_test: 0.520
Epoch [ 4/5]. Step [201/391]. Loss_valid: 0.0322  Acc_valid: 0.542  Loss_test: 0.0326  Acc_test: 0.522
Epoch [ 5/5]. Step [  1/391]. Loss_valid: 0.0321  Acc_valid: 0.551  Loss_test: 0.0326  Acc_test: 0.526
Epoch [ 5/5]. Step [201/391]. Loss_valid: 0.0321  Acc_valid: 0.556  Loss_

## CNN на CIFAR-100 через дообучение ImageNet Resnet-50 с аугментацией данных.

In [25]:
import albumentations as alb
from albumentations.pytorch import ToTensorV2

alb_transform_train = alb.Compose(
    [
        alb.Resize(height=40, width=40),
        alb.RandomCrop(height=32, width=32),
        alb.HorizontalFlip(p=0.5),
        alb.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=1),
        alb.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        alb.RandomBrightnessContrast(p=0.5),
        alb.OneOf([
            alb.MotionBlur(p=1),
            alb.OpticalDistortion(p=1),
            alb.GaussNoise(p=1)                 
        ], p=1),
        alb.Normalize(mean=(0.485, 0.456, 0.406),
                      std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

alb_transform_test = alb.Compose(
    [   alb.Normalize(mean=(0.485, 0.456, 0.406),
                      std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

In [26]:
class Net_resnet_alb(nn.Module):
    def __init__(self, resnet50, device='cpu'):
        super().__init__()
        self.device = device
        self.net = resnet50
        for param in list(self.net.parameters())[:]:
            param.requires_grad = False
        self.net.fc = nn.Sequential(            
            nn.Linear(2048, 2048),
            nn.LeakyReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.25),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.25),
            nn.Linear(1024, 100),
            nn.Softmax(dim=1)
            )

    def forward(self, x):
        x = self.net(x)
        return x


    def fit(self, loader_train, loadre_test, epochs, loss, optimazer, lr, show_log=True):
        self.train()
        params_to_update = []
        for name, param in self.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
        self.history = []
        self.optimazer = optimazer(params_to_update, lr)    
        self.loss = loss()
        for epoch in range(epochs):
            running_loss = 0.0
            running_items = 0.0
            predict_ture = 0.0            
            for i, data in enumerate(loader_train):
                X_train, y_train = data[0]['image'].to(self.device), data[1].to(self.device)
                self.optimazer.zero_grad()
                outputs = self(X_train)
                loss_iter = self.loss(outputs, y_train)
                loss_iter.backward()
                self.optimazer.step()
                if i%200 ==0:
                    loss_train, acc_train = self.loss_fn(loader_train)
                    loss_test, acc_test = self.loss_fn(loader_test)
                    self.history.append([epoch+1, i+1, acc_train, acc_test])
                    if show_log:
                        print(f'Epoch [{epoch + 1:2}/{epochs}]. ' \
                              f'Step [{i + 1:3}/{len(loader_train)}]. ' \
                              f'Loss_valid: {loss_train:.4f}  '\
                              f'Acc_valid: {acc_train:.3f}  '\
                              f'Loss_test: {loss_test:.4f}  '\
                              f'Acc_test: {acc_test:.3f}'
                              )
                running_loss, running_items = 0.0, 0.0
        print(f'Training is finished with optimazer: {optimazer}')
        return self.history


    def loss_fn(self, loader):
        self.eval()
        running_loss = 0.0
        running_items = 0.0
        predict_ture = 0.0    
        for i, data in enumerate(loader):
            X_data, y_data = data[0]['image'].to(self.device), data[1].to(self.device)
            outputs = self(X_data)
            loss_iter = self.loss(outputs, y_data)
            running_loss += loss_iter.item()
            _, predicted = torch.max(outputs, 1)
            predict_ture += (y_data==predicted).sum().item()
            running_items += len(y_data)
        self.train()  
        return running_loss / running_items, predict_ture/running_items

In [27]:
class MyOwnCifar_alb(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
             img = self.transform(image=np.array(img))
        return img, self._base_dataset[idx][1]

In [28]:
train_dataset = MyOwnCifar_alb(train_dataset_cifar100, alb_transform_train)
test_dataset = MyOwnCifar_alb(test_dataset_cifar100, alb_transform_test)

In [29]:
loader_train = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=2)
loader_test = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=128,
                                           shuffle=False,
                                           num_workers=1)

In [30]:
net = Net_resnet_alb(resnet50, device).to(device)

In [31]:
hist = net.fit(loader_train, loader_test, 5, nn.CrossEntropyLoss, optim.Adam, 0.0001, show_log=True)

Epoch [ 1/5]. Step [  1/391]. Loss_valid: 0.0360  Acc_valid: 0.010  Loss_test: 0.0364  Acc_test: 0.010
Epoch [ 1/5]. Step [201/391]. Loss_valid: 0.0353  Acc_valid: 0.120  Loss_test: 0.0356  Acc_test: 0.125
Epoch [ 2/5]. Step [  1/391]. Loss_valid: 0.0350  Acc_valid: 0.153  Loss_test: 0.0354  Acc_test: 0.156
Epoch [ 2/5]. Step [201/391]. Loss_valid: 0.0349  Acc_valid: 0.168  Loss_test: 0.0352  Acc_test: 0.172
Epoch [ 3/5]. Step [  1/391]. Loss_valid: 0.0348  Acc_valid: 0.180  Loss_test: 0.0352  Acc_test: 0.182
Epoch [ 3/5]. Step [201/391]. Loss_valid: 0.0347  Acc_valid: 0.192  Loss_test: 0.0351  Acc_test: 0.190
Epoch [ 4/5]. Step [  1/391]. Loss_valid: 0.0347  Acc_valid: 0.193  Loss_test: 0.0351  Acc_test: 0.191
Epoch [ 4/5]. Step [201/391]. Loss_valid: 0.0347  Acc_valid: 0.199  Loss_test: 0.0350  Acc_test: 0.195
Epoch [ 5/5]. Step [  1/391]. Loss_valid: 0.0346  Acc_valid: 0.203  Loss_test: 0.0350  Acc_test: 0.197
Epoch [ 5/5]. Step [201/391]. Loss_valid: 0.0346  Acc_valid: 0.206  Loss_

## Замечания по работе

Низкое качество сети с аугментацией данных вызвано моей ошибкой. Первоначально я работал с  первой «самописной» сетью, в которой входной изображение 32*32. И в последствии забыл изменить строки:
- alb.Resize(height=40, width=40),
- alb.RandomCrop(height=32, width=32),<br>
на 
- alb.Resize(height= 256, width= 256),
- alb.RandomCrop(height=224, width=224),<br>
В силу низкой производительности моего компьютера, и наступившего ограничения по использованию GPU на сервисе GoogleColab, переучивать сеть не стал. 
